In [26]:
# GUI for jupyter: https://docs.bamboolib.8080labs.com/

import requests
from config import AV_API_KEY

STOCK_SYMBOL = 'INTC'
AV_URL = "https://www.alphavantage.co/query" # Alpha Vantage API call: 


def get_from_av(sym, query_type):
    return requests.get(AV_URL, params={
        "function": query_type,
        "symbol": sym,
        "apikey": AV_API_KEY
    }).json()

def get_stock_data(sym):
    '''retrieve data from Alpha Vantage: OVERVIEW, BALANCE_SHEET, TIME_SERIES_MONTHLY_ADJUSTED'''
    data = {}
    query_types = []
    data['overview'] = get_from_av(sym, 'OVERVIEW')
    data['annual_reports'] = get_from_av(sym, 'BALANCE_SHEET')['annualReports']
    data['monthly_quotes'] = get_from_av(sym, 'TIME_SERIES_MONTHLY_ADJUSTED')
    return data

data = None
data = get_stock_data(STOCK_SYMBOL)

0YDGH75EGBYCCRBQ


In [20]:
print(data['overview'])

{'Symbol': 'INTC', 'AssetType': 'Common Stock', 'Name': 'Intel Corporation', 'Description': 'Intel Corporation provides computing, networking, data storage, and communication solutions worldwide. It operates through Data Center Group, Internet of Things Group, Non-Volatile Memory Solutions Group, Programmable Solutions Group, Client Computing Group, and All Other segments. The company offers platform products, such as CPU and chipset, system-on-chip, and multichip package products for cloud, enterprise, and communication infrastructure markets. It also provides NAND flash memory and DC persistent products for enterprise and cloud-based data centers, and users of business and consumer desktops and laptops; programmable semiconductors, such as field-programmable gate arrays, application-specific integrated circuits, and related products for communications, data center, industrial, and military markets; and various processors for notebooks, mobiles, and desktop PCs. In addition, it offers

In [21]:
data['overview'].keys()

dict_keys(['Symbol', 'AssetType', 'Name', 'Description', 'Exchange', 'Currency', 'Country', 'Sector', 'Industry', 'Address', 'FullTimeEmployees', 'FiscalYearEnd', 'LatestQuarter', 'MarketCapitalization', 'EBITDA', 'PERatio', 'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS', 'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM', 'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM', 'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'TrailingPE', 'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue', 'EVToEBITDA', 'Beta', '52WeekHigh', '52WeekLow', '50DayMovingAverage', '200DayMovingAverage', 'SharesOutstanding', 'SharesFloat', 'SharesShort', 'SharesShortPriorMonth', 'ShortRatio', 'ShortPercentOutstanding', 'ShortPercentFloat', 'PercentInsiders', 'PercentInstitutions', 'ForwardAnnualDividendRate', 'ForwardAnnualDividendYield', 'PayoutRatio', 'DividendDate', 'ExDividendDate', 'LastS

In [22]:
# Displays latests Annual Report
print(data['annual_reports'][0])

{'fiscalDateEnding': '2019-12-31', 'reportedCurrency': 'USD', 'totalAssets': '136524000000', 'intangibleAssets': '10827000000', 'earningAssets': 'None', 'otherCurrentAssets': '46977000000', 'totalLiabilities': '57497000000', 'totalShareholderEquity': '77504000000', 'deferredLongTermLiabilities': '2044000000', 'otherCurrentLiabilities': '10636000000', 'commonStock': '25261000000', 'retainedEarnings': '53523000000', 'otherLiabilities': '11027000000', 'goodwill': '26276000000', 'otherAssets': '4925000000', 'cash': '3276000000', 'totalCurrentLiabilities': '22310000000', 'shortTermDebt': '3693000000', 'currentLongTermDebt': '3693000000', 'otherShareholderEquity': '-1280000000', 'propertyPlantEquipment': '56014000000', 'totalCurrentAssets': '31239000000', 'longTermInvestments': '7243000000', 'netTangibleAssets': '40401000000', 'shortTermInvestments': '8929000000', 'netReceivables': '13212000000', 'longTermDebt': '25308000000', 'inventory': '8744000000', 'accountsPayable': '7981000000', 'tota

In [23]:
data['annual_reports'][0].keys()

dict_keys(['fiscalDateEnding', 'reportedCurrency', 'totalAssets', 'intangibleAssets', 'earningAssets', 'otherCurrentAssets', 'totalLiabilities', 'totalShareholderEquity', 'deferredLongTermLiabilities', 'otherCurrentLiabilities', 'commonStock', 'retainedEarnings', 'otherLiabilities', 'goodwill', 'otherAssets', 'cash', 'totalCurrentLiabilities', 'shortTermDebt', 'currentLongTermDebt', 'otherShareholderEquity', 'propertyPlantEquipment', 'totalCurrentAssets', 'longTermInvestments', 'netTangibleAssets', 'shortTermInvestments', 'netReceivables', 'longTermDebt', 'inventory', 'accountsPayable', 'totalPermanentEquity', 'additionalPaidInCapital', 'commonStockTotalEquity', 'preferredStockTotalEquity', 'retainedEarningsTotalEquity', 'treasuryStock', 'accumulatedAmortization', 'otherNonCurrrentAssets', 'deferredLongTermAssetCharges', 'totalNonCurrentAssets', 'capitalLeaseObligations', 'totalLongTermDebt', 'otherNonCurrentLiabilities', 'totalNonCurrentLiabilities', 'negativeGoodwill', 'warrants', 'p

In [31]:
# Calculate Historical Fair Value (11 years) from:
# 5-yr average yield
# 12-yr median yield
# Earnings
# Owner earnings
# Operating cash flow
# Free cash flow
# EBITDA
# EBIT
## Then average these all together

# calculate equity value from EV/EBITDA
# https://www.wallstreetoasis.com/forums/evebidta-to-target-price#:~:text=With%20the%20EV%2FEBITDA%20multiple,the%20equity%20value%20per%20share.
enterprise_value = float(data['overview']['EVToEBITDA']) * float(data['overview']['EBITDA'])
total_debt = float(data['annual_reports'][0]['shortTermDebt']) + float(data['annual_reports'][0]['longTermDebt'])
equity_value = float(enterprise_value) - float(total_debt)
share_price = round(float(equity_value) / float(data['overview']['SharesOutstanding']), 2)
print("SHARE PRICE from EBITDA: ", share_price) # NOTE: SEEMS TO BE CURRENT YEAR. 

#TODO: Calculate share price from EBITDA for past 11-yrs

SHARE PRICE from EBITDA:  43.33
